In [33]:
# Import necessary libraries
import pandas as pd

# File paths for all datasets (replace with actual paths when running the code)
base_path = r"C:\Users\pumo1001\Desktop\All databases\BF_CFS_IT_20221102\BF_CFS_IT_20221102\\"
dimension_file = base_path + "BF_CFS_IT_DIM.CSV"
fact_file = base_path + "BF_CFS_IT_FCT.CSV"
market_file = base_path + "BF_CFS_IT_MKT.CSV"
period_file = base_path + "BF_CFS_IT_PER1.CSV"
product_file = base_path + "BF_CFS_IT_PROD.CSV"
fact_data_file = base_path + "BF_CFS_IT_fact_data.CSV"
reference_file = base_path + "QCparameters.CSV"
market_mapping_file = base_path + "Market_mapping_it.csv"
fact_mapping_file = base_path + "Fact_mapping_it.csv"
characteristics_file = "product_characteristics.csv" 
global_code_list_file = "global_code_list.csv" 


# Load data into pandas dataframes
dimension_df = pd.read_csv(dimension_file, sep='|')
fact_df = pd.read_csv(fact_file, sep='|')
market_df = pd.read_csv(market_file, sep='|')
period_df = pd.read_csv(period_file, sep='|')
product_df = pd.read_csv(product_file, sep='|')
data_df = pd.read_csv(fact_data_file, sep='|')
reference_df = pd.read_csv(reference_file)
market_mapping_df = pd.read_csv(market_mapping_file) 
fact_mapping_df = pd.read_csv(fact_mapping_file) 


# Function to dynamically load files from a given path
def load_file(file_path, delimiter='|'):
    return pd.read_csv(file_path, sep=delimiter)

# -----------------------------------
# QC Check: Number of Periods
def validate_periods(period_df):
    num_periods = len(period_df)
    valid = 36 <= num_periods <= 38
    return valid, num_periods

# QC Check: File Size (number of products within range)
def validate_file_size(product_df):
    num_products = len(product_df)
    min_size = 0.8 * 100  # Adjust based on expected size
    max_size = 1.2 * 100
    return min_size <= num_products <= max_size, num_products

# QC Check: Missing reference data
def check_missing_reference(df, column_name):
    missing = df[column_name].isnull().sum()
    return missing == 0, missing

# QC Check: Global Code consistency between input and output
def check_global_code_consistency(input_df, output_df):
    input_codes = set(input_df['GlobalCode'].unique())
    output_codes = set(output_df['GlobalCode'].unique())
    return input_codes == output_codes, len(input_codes)

# QC Check: Hierarchy Presence
def check_hierarchy_presence(df, hierarchy_columns):
    missing_cols = [col for col in hierarchy_columns if col not in df.columns]
    return len(missing_cols) == 0, missing_cols

# QC Check: Fact data integrity
def validate_fact_data(fact_df):
    # Example: Check if all values are non-negative
    invalid_entries = fact_df.select_dtypes(include=['number']).lt(0).sum().sum()
    return invalid_entries == 0, invalid_entries

'''# QC Check for Missing/Unmatched Items
def qc_check_missing_unmatched(reference_df, data_df, market_df, product_df, period_df, fact_df):
    # 1. Market Check (compare MKT_TAG with Market)
    unmatched_market = reference_df[~reference_df['MKT_TAG'].isin(market_df['TAG'])]
    print(f"Unmatched Markets:\n{unmatched_market['MKT_TAG'].dropna().unique()}")

    # 2. Product Check (compare PROD_TAG with Product)
    unmatched_product = reference_df[~reference_df['PROD_TAG'].isin(product_df['TAG'])]
    print(f"Unmatched Products:\n{unmatched_product['PROD_TAG'].dropna().unique()}")

    # 3. Period Check (compare PER_TAG with Period)
    unmatched_period = reference_df[~reference_df['PER_TAG'].isin(period_df['TAG'])]
    print(f"Unmatched Periods:\n{unmatched_period['PER_TAG'].dropna().unique()}")

    # 4. Fact Check (compare Fact columns with Fact file)
    unmatched_fact = reference_df[~reference_df['Fact'].isin(fact_df['TAG'])]
    print(f"Unmatched Facts:\n{unmatched_fact['Fact'].dropna().unique()}")
'''

# Merge fact_data with market, product, and period dimension tables
merged_data = pd.merge(data_df, market_df, left_on='MKT_TAG', right_on='TAG', how='left')
merged_data = pd.merge(merged_data, product_df, left_on='PROD_TAG', right_on='TAG', how='left')
merged_data = pd.merge(merged_data, period_df, left_on='PER_TAG', right_on='TAG', how='left')

# Melt the fact_data to unpivot the fact columns
melted_data = pd.melt(merged_data, 
                      id_vars=['MKT_TAG', 'PROD_TAG', 'PER_TAG'], 
                      value_vars=['M0000001', 'M0000002', 'M0000003', 'M0000004', 
                                  'M0000005', 'M0000006', 'M0000007', 'M0000008', 
                                  'M0000009', 'M0000010', 'M0000011'],
                      var_name='TAG', 
                      value_name='fact_value')

# Merge with fact metadata (fact_df)
merged_fact_data = pd.merge(melted_data, fact_df, on='TAG', how='left')

# Run QC checks on markets, products, periods, and facts
qc_check(data_df, market_df, 'TAG', 'MKT_TAG', 'markets')
#qc_check(data_df, product_df, 'TAG', 'PROD_TAG', 'products')
qc_check(data_df, period_df, 'TAG', 'PER_TAG', 'periods')
fact_qc_check(data_df, fact_df)

# Example QC checks:
period_check, num_periods = validate_periods(period_df)
file_size_check, num_products = validate_file_size(product_df)
fact_integrity_check, invalid_facts = validate_fact_data(data_df)

# Output the QC results
qc_results = {
    'period_check': period_check,
    'num_periods': num_periods,
    'file_size_check': file_size_check,
    'num_products': num_products,
    'fact_integrity_check': fact_integrity_check,
    'invalid_facts': invalid_facts
}

'''# Market Transformation
def transform_market(market_df, market_mapping_df):
    # Merge on MCODE
    enriched_market_df = pd.merge(market_df, market_mapping_df, left_on='TAG', right_on='Local Tag', how='left')
    return enriched_market_df

# Fact Transformation
def transform_fact(fact_df, fact_mapping_df):
    # Merge on Local Tag
    enriched_fact_df = pd.merge(fact_df, fact_mapping_df, left_on='TAG', right_on='Local Tag', how='left')
    return enriched_fact_df'''

# product transformation
def transform_product(product_df, characteristics_file, global_code_list_file, base_path):
    # Load the characteristic mapping and global code list files
    characteristics_df = pd.read_csv(base_path + characteristics_file)
    global_code_df = pd.read_csv(base_path + global_code_list_file)

    # 1. Mapping Characteristics: Map input columns to output columns
    for index, row in characteristics_df.iterrows():
        if pd.notnull(row['ALIAS_NAME']) and row['ALIAS_NAME'].startswith('Input'):
            input_column = row['ALIAS_NAME']
            output_column = row['NAME']
            charname = row['GCLCHARNAME']
            
            # 2. Code to Value Transformation: Convert codes to values based on GCLCHARNAME
            if charname in global_code_df['CHARNAME'].unique():
                mapping = global_code_df[global_code_df['CHARNAME'] == charname][['GLOBALCODE', 'CHARVALUE']]
                
                # Replace the product code with the corresponding value
                product_df[output_column] = product_df[input_column].map(
                    dict(zip(mapping['GLOBALCODE'], mapping['CHARVALUE']))
                )
    
    print("Product Transformation Completed:")
    return product_df


# Perform transformations
#enriched_market_df = transform_market(market_df, market_mapping_df)
#enriched_fact_df = transform_fact(fact_df, fact_mapping_df)
product_df_transformed = transform_product(product_df, characteristics_file, global_code_list_file, base_path)

'''# Display results (first few rows for validation)
print("Enriched Market Data:")
print(enriched_market_df.head())

print("\nEnriched Fact Data:")
print(enriched_fact_df.head())'''


#print(qc_results)

# Display a sample of the merged data
#print(merged_fact_data.head())


Unmatched markets:
[]
Missing markets:
[]
Unmatched periods:
[]
Missing periods:
[]
Unmatched facts:
['MKT_TAG']
Missing facts:
[]
Product Transformation Completed:


'# Display results (first few rows for validation)\nprint("Enriched Market Data:")\nprint(enriched_market_df.head())\n\nprint("\nEnriched Fact Data:")\nprint(enriched_fact_df.head())'

In [47]:
#product_df_transformed.loc['TAG','SHORT','Global Manufacturer','Global Subbrand']
product_df_transformed.tail(100)

,TAG,SHORT,LONG,DISPLAY_ORDER,PARENT_TAG,HIER_NUM,HIER_NAME,HIER_LEVEL_NUM,HIER_LEVEL_NAME,Global Super Category,Global Category,Global Segment,Global Sub Segment,Global Manufacturer,Global Brand Family,Global Brand,Global Subbrand,Ean13,Local Brand,Local Manufacturer
8152,H0048348,SIMMNTHL,SIMMNTHL,8153,H0048270,4,H004 CFS TOTAL,5,Global Subbrand,CFS,NaN,NaN,NaN,BOLTGROP,SIMMNTHL,SIMMNTHL,SIMMNTHL,NaN,NaN,NaN
8153,H0048349,SPUNTI,SPUNTI,8154,H0048271,4,H004 CFS TOTAL,5,Global Subbrand,CFS,NaN,NaN,NaN,BOLTGROP,SIMMNTHL,SPUNTI,SPUNTI,NaN,NaN,NaN
8154,H0048350,CALLCAPR,CALLCAPR,8155,H0048272,4,H004 CFS TOTAL,5,Global Subbrand,CFS,NaN,NaN,NaN,CALLIPO,CALLIPO,CALLIPO,CALLCAPR,NaN,NaN,NaN
8155,H0048351,CALLERUS,CALLERUS,8156,H0048272,4,H004 CFS TOTAL,5,Global Subbrand,CFS,NaN,NaN,NaN,CALLIPO,CALLIPO,CALLIPO,CALLERUS,NaN,NaN,NaN
8156,H0048352,CALLIPO,CALLIPO,8157,H0048272,4,H004 CFS TOTAL,5,Global Subbrand,CFS,NaN,NaN,NaN,CALLIPO,CALLIPO,CALLIPO,CALLIPO,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8247,H0048441,MARVENAT,MARVENAT,8248,H0048321,4,H004 CFS TOTAL,5,Global Subbrand,CFS,NaN,NaN,NaN,THAIUNIN,MAREBLU,MAREBLU,MARVENAT,NaN,NaN,NaN
8248,H0048442,MARVESAP,MARVESAP,8249,H0048321,4,H004 CFS TOTAL,5,Global Subbrand,CFS,NaN,NaN,NaN,THAIUNIN,MAREBLU,MAREBLU,MARVESAP,NaN,NaN,NaN
8249,H0048443,MARWEWAT,MARWEWAT,8250,H0048321,4,H004 CFS TOTAL,5,Global Subbrand,CFS,NaN,NaN,NaN,THAIUNIN,MAREBLU,MAREBLU,MARWEWAT,NaN,NaN,NaN
8250,H0048444,MRBUETSO,MRBUETSO,8251,H0048321,4,H004 CFS TOTAL,5,Global Subbrand,CFS,NaN,NaN,NaN,THAIUNIN,MAREBLU,MAREBLU,MRBUETSO,NaN,NaN,NaN
